In [ ]:
%matplotlib inline
#%pylab
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.colors as mpc
import matplotlib.dates as dts
import numpy as np
import pandas as pd
import itertools
import os
import ROOT
import datetime
from root_numpy import root2array, root2rec, tree2rec, array2root
from scipy.optimize import curve_fit
from scipy.misc import factorial
plt.rcParams.update({'font.size': 16})
#from larlite import larlite
#k=larlite.trigger()
from ROOT import larlite
larlite.trigger()
from ROOT import trigger
from fememu_pycfg import apply_config


In [ ]:
# emulator configuration
cfg_file = 'debug_data.cfg'
config = trigger.fememu.FEMBeamTriggerConfig()
apply_config(config,cfg_file)
# emulator construction
emu = trigger.fememu.LLInterface(config)
# data file loading
f_trig = ROOT.TFile('~/uBooNE/data/3rdFEM/Dec16/run005/test123_pt_trig_m.root')
f_xmit = ROOT.TFile('~/uBooNE/data/3rdFEM/Dec16/run005/test123_pt_trig.root')
t_trig = f_trig.Get('trigger_daq_tree')
t_xmit = f_xmit.Get('fifo_pmt_xmit_tree')
print 'trigger entries :',t_trig.GetEntries()
print 'xmit entries    :',t_xmit.GetEntries()


In [ ]:
trig_times = {}
for x in xrange(t_trig.GetEntries()):
    t_trig.GetEntry(x)
    trig = t_trig.trigger_daq_branch
    trig_num  = trig.TriggerNumber()
    trig_time = trig.TriggerTime()
    trig_frame = int(trig_time /1600.)
    trig_sample = (trig_time - trig_frame * 1600.) / (1./64.)
    trig_times[trig.TriggerNumber()] = (trig_time,trig_frame,trig_sample)

# find first saturation tick number
def getFireThresh(vec,thresh_hold):
    for idx in xrange(len(vec)):
        if vec[idx] >= thresh_hold:
            return idx
    return -1

def getFireTick(vec):
    for idx in xrange(len(vec)):
        if idx < 4: continue
        if vec[idx] - vec[idx-4] >= 10: 
            return idx
    return -1

# find first saturation tick number
def getFireTick200(vec):
    for idx in xrange(len(vec)):
        if idx < 4: continue
        if vec[idx] - vec[idx-4] >= 200: 
            return idx
    return -1
        
def getFireTick2000(vec):
    for idx in xrange(len(vec)):
        if idx < 4: continue
        if vec[idx] - vec[idx-4] >= 2000: 
            return idx
    return -1

In [ ]:
# loop through xmit info

# tick at which the firmware trigger fires
fire_time_v = []
# tick at which the DIFF3 vector is maximal
diff3_max_tick_v = []

show_ch1 = False
print t_xmit.GetEntries()

trig_times_v = []

gate_start = 69
gate_end   = 172

for x in xrange(t_xmit.GetEntries()):
#for x in xrange(1000):
    
    t_xmit.GetEntry(x)
    #t_trig.GetEntry(x)
    
    ev_fifo = t_xmit.fifo_pmt_xmit_branch
    frame_num = sample_num = 0
    
    ch0,ch1,beamch=(None,None,None)
    
    wf_v = np.zeros(gate_end-gate_start)

    for n in xrange(ev_fifo.size()):
        fifo = ev_fifo[n]
        if not fifo.module_address() == 4: continue
        channel = fifo.channel_number()
        if channel < 32:
            wf_v += np.array(fifo[gate_start:gate_end])-2048  
        if channel == 0: 
            ch0 = fifo
        elif channel == 1:
            ch1 = fifo
        elif channel in [47]:
            beamch = fifo

    ch0_adc = np.array(ch0)
    ch1_adc = np.array(ch1)
    beamch_adc = np.array(beamch)

    # Get timings
    beamgate_sample = ch0.readout_sample_number_RAW()
    
    trig_num = ev_fifo.event_number()
    
    trig_time,trig_frame,trig_sample = trig_times[trig_num]
    trig_tick = trig_sample - beamgate_sample
    
    # adjust for frame alignment
    if (trig_tick < -1000):
        trig_tick += 102400
    
    if (trig_tick > 500 or trig_tick < 0):
        continue
    
    if (trig_tick < 500):
        trig_times_v.append(trig_tick)
        
    #continue

    beamch_tick = beamch.readout_sample_number_RAW() - beamgate_sample
    beamch_fire = getFireThresh(beamch_adc,3000)
            
    # get saturation tick for ch1
    ch0_fire = getFireTick(ch0_adc)
    #if ch0_fire in fire_time_v: continue
        
    fire_time_v.append(ch0_fire)  

    print 'BeamCh Start     @', beamch_tick
    print 'BeamCh PHMax>100 @', beamch_tick + beamch_fire
    print 'Ch. 0 fire       @', ch0_fire
    print 'Trigger fire     @', trig_tick
    print 'Diff             @', trig_tick-ch0_fire
    # Emulator
    emu_out = emu.Emulate(ev_fifo)
    print 'Emulator fire    @',
    for t in emu_out.fire_time_v:
        print t,
    print
    
    #'''
    
    disc3_vecs=[]
    phmax_vecs=[]
    
    phmax_tot = np.zeros(gate_end-gate_start)
    phmax_max_v = np.zeros(gate_end-gate_start)
    
    phmax_max_pmt_num = -1
    phmax_max_pmt_adc = -1
    
    disc3_tot = np.zeros(gate_end-gate_start)
    disc3_max_v = np.zeros(gate_end-gate_start)
    
    disc3_max_pmt_num = -1
    disc3_max_pmt_adc = -1
    
    for y in range(32):
        
        phmax_vecs.append(emu.ChannelPHMAXVector(y))
        disc3_vecs.append(emu.PhaseDiffDisc3(y))
        diff_0 = emu.PhaseDiffDisc0(y)
        disc0_fire = getFireThresh(diff_0,10)
        
        phmax_pmt = np.array(phmax_vecs[y][gate_start:gate_end])
        max_adc = np.amax(phmax_pmt)
        if (max_adc > phmax_max_pmt_adc):
            phmax_max_pmt_num = y
            phmax_max_pmt_adc = max_adc
            phmax_max_v = phmax_pmt
        phmax_tot += phmax_pmt
        
        disc3_pmt = np.array(disc3_vecs[y][gate_start:gate_end])
        max_adc = np.amax(disc3_pmt)
        if (max_adc > disc3_max_pmt_adc):
            disc3_max_pmt_num = y
            disc3_max_pmt_adc = max_adc
            disc3_max_v = disc3_pmt
        disc3_tot += disc3_pmt
        
        if(disc0_fire==-1):
            disc0_fire = 0
        diff_3 = emu.PhaseDiffDisc3(y)
        disc3_fire = getFireThresh(diff_3,200)
        if(disc3_fire==-1):
            disc3_fire = 0
    
    #'''
    
    diff3_max_tick_v.append(np.argmax(disc3_tot))
    
    xticks = np.linspace(gate_start,gate_end,gate_end-gate_start)
    
    fig = plt.figure(figsize=(15,6))
    plt.plot(xticks,disc3_tot,label='DIFF3 VEC TOT')
    plt.plot(xticks,disc3_max_v,label='DIFF3 vec [pmt %02i]'%phmax_max_pmt_num)
    plt.axvline(trig_tick,color='r',lw=2,label='Trigger (Tick %d)' % trig_tick)
    plt.grid()
    plt.legend()
    #plt.xlim([0,200])
    plt.show()
    
    fig = plt.figure(figsize=(15,6))
    plt.plot(xticks,wf_v)
    plt.axvline(trig_tick,color='r',lw=2,label='Trigger (Tick %d)' % trig_tick)
    #plt.xlim([0,200])
    plt.grid()
    plt.show()
    
    #'''
    
fig = plt.figure(figsize=(10,6))
plt.hist(trig_times_v,bins=np.linspace(50,200,40))
plt.grid()
plt.show()
print 'min trigger tick: ',np.amin(trig_times_v)
print 'max trigger ticl: ',np.amax(trig_times_v)

fig = plt.figure(figsize(10,6))
plt.hist(diff3_max_tick_v,bins=np.linspace(0,100,10))
plt.grid()
plt.show()


In [ ]:
fig = plt.figure(figsize=(10,6))
plt.hist(diff3_max_tick_v,bins=np.linspace(0,100,20))
plt.grid()
plt.show()